<a href="https://colab.research.google.com/github/biggymart/colab/blob/main/ChaptchaRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=IcLEJB2pY2Y&t=1052s

Title: Captcha recognition using PyTorch (Convolutional-RNN + CTC Loss)

Book: Approaching (Almost) Any Machine Learning Problem (Abhishek Thakur)

In [ ]:
# config.py

DATA_DIR = '../input/captcha_images_v2'
BATCH_SIZE = 8
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 75
NUM_WORKERS = 8
EPOCHS = 200
DEVICE = "cuda"

In [ ]:
import albumentations
import torch
import numpy as np

from PIL import Image
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

class Classification:
  def __init__(self, image_paths, targets, resize=None):
    self.